In [1]:
import pandas as pd
from tqdm import tqdm
from functools import reduce
import numpy as np
import time
import re

#Data Ingestion to Neo4j
from py2neo import Graph
from py2neo import Node,Relationship
from py2neo import NodeMatcher

import pandas as pd
import numpy as np
import json
from urllib.request import urlopen
import json
from tqdm import tqdm
import time
import numpy as np

# GET DATASET

In [43]:
df= pd.read_csv('GT_NEW_07132021.csv')
df['content_id'] =  df['filename'].str.extract('(\d+)').astype(int)
df = df.rename(columns={"sic": "SIC_Num"})
df

,sic,lob,Relation,filename,extracted_text
0,['5812'],BOP,Exposures,rx69133.pdf,Ineligible Classes/Exposures \nGeneral BOP Ine...
1,['7997'],General Liability,Loss Insight,rx47638.pdf,This claim illustrates the critical nature of ...
2,['7997'],Property,NaN,rx70211.pdf,NOTE: Include all buildings within 100 feet of...
3,['7997'],General Liability,Exposures,rx70211.pdf,DO NOT WRITE golf clubs with the following exp...
4,['7997'],Auto,Exposures,rx70211.pdf,Refer to EFU if:\n• Any liability deductibles\...
5,['7997'],Worker's Compensation,Exposures,rx70211.pdf,Golf Clubs are not a class of business we acti...
6,['7997'],General Liability,Limits,rx70211.pdf,$1mm occurrence/$2mm General Aggregate/$2mm Pr...
7,['7997'],Auto,Limits,rx70211.pdf,• $1mm Liability\n• Maximum physical damage li...
8,['7812'],Film Package,Coverages,rx22627.pdf,The main coverages are:\nCast – proves for ext...
9,['7812'],General Liability,Coverages,rx22627.pdf,General Liability coverage is normally provide...


In [80]:
df = pd.read_excel('Non Commercial Research Org_07_19.xlsx')
df = df.loc[:,['Submission Doc','SIC_Num','LoB','Relation.1','filename','Extracted text']]
df = df.rename(columns={"Submission Doc": "submission_doc", "Relation.1": "Relation", "LoB":"lob", "Extracted text":"extracted_text"})
df['content_id'] =  df['filename'].str.extract('(\d+)').astype(int)
df.head()

,submission_doc,SIC_Num,lob,Relation,filename,extracted_text,content_id
0,P318091211495676.pdf,8733,General Liability,Exposures,rx70252.pdf,The following Life Sciences SIC Codes are Low ...,70252
1,P318091211495676.pdf,8733,Property,Exposures,rx70252.pdf,The following Life Sciences SIC Codes are Low ...,70252
2,P318091211495676.pdf,8733,General Liability,CRFs,rx63069.pdf,"Laboratories – SIC 8071, 8072, 8733, 8734, 873...",63069
3,P318091211495676.pdf,8733,Property,CRFs,rx63069.pdf,"Laboratories – SIC 8071, 8072, 8733, 8734, 873...",63069
4,P318091211495676.pdf,8733,General Liability,Exposures,rx63069.pdf,Primary Underwriting Considerations • Microbio...,63069


In [102]:
df

,submission_doc,SIC_Num,lob,Relation,filename,extracted_text,content_id
0,P318091211495676.pdf,8733,General Liability,Exposures,rx70252.pdf,The following Life Sciences SIC Codes are Low ...,70252
1,P318091211495676.pdf,8733,Property,Exposures,rx70252.pdf,The following Life Sciences SIC Codes are Low ...,70252
2,P318091211495676.pdf,8733,General Liability,CRFs,rx63069.pdf,"Laboratories – SIC 8071, 8072, 8733, 8734, 873...",63069
3,P318091211495676.pdf,8733,Property,CRFs,rx63069.pdf,"Laboratories – SIC 8071, 8072, 8733, 8734, 873...",63069
4,P318091211495676.pdf,8733,General Liability,Exposures,rx63069.pdf,Primary Underwriting Considerations • Microbio...,63069
5,P318091211495676.pdf,8733,Property,Exposures,rx63069.pdf,Primary Underwriting Considerations • Microbio...,63069
6,P318091211495676.pdf,SIC Agnostic,Property,Risk Class,76207.pdf,When determining the appropriate risk class fo...,76207
7,P318091211495676.pdf,8733,Property,Limits,66176.pdf,Description: \nProperty/Business Income Global...,66176
8,P318091211495676.pdf,8733,General Liability,Guidelines,66176.pdf,Description:\nIncrease non-owned watercraft in...,66176
9,P318091211495676.pdf,SIC Agnostic,General Liability,Underwriting Strategy,2435.pdf,A Privacy exclusion or an Access or Disclosure...,2435


In [103]:
df.content_id.unique()

array([70252, 63069, 76207, 66176,  2435, 51665])

In [22]:
#metadata
metadata = pd.read_excel('ChubbnetExtract3_010121_FixedURL.xlsx')
metadata.head()

,SBU Code,SBU Name,SiteID,Site Name,Primary Cat ID,Primary Category,Secondary Cat ID,Secondary Category,Tertiary Cat ID,Tertiary Category,...,Reviewer Last Name,Form Number,Edition Text,Publishing Instructions,Document Version,Content Creator ID,Content Creator First Name,Content Creator Last Name,Content Creator Email,Workflow State Change Time
0,ci,Commercial Insurance,om,Ocean Cargo,18,Strategy,15,Underwriting Guidelines,6,Strategies / Guidelines,...,Hummel,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ci,Commercial Insurance,om,Ocean Cargo,18,Strategy,15,Underwriting Guidelines,6,Strategies / Guidelines,...,Hummel,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ci,Commercial Insurance,ht,Healthcare Information Tech.,2,Applications,0,No Secondary Category,0,No Tertiary Category,...,Somarriba,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ci,Commercial Insurance,li,Life Sciences,2,Applications,0,No Secondary Category,0,No Tertiary Category,...,Somarriba,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ci,Commercial Insurance,li,Life Sciences,2,Applications,0,No Secondary Category,0,No Tertiary Category,...,Ballantyne,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
metadata_dedup = metadata.drop_duplicates(subset=['Document File Name'])
metadata_dedup = metadata_dedup.rename(columns={"Content Id":'content_id'})
metadata_dedup['content_id'] = metadata_dedup['content_id'].astype(int)
metadata_dedup.shape

(21458, 49)

In [29]:
metadata_required = metadata_dedup.loc[:,['content_id', 'Content Title','Content Description', 'Content State','SBU Code', 'SBU Name', 'Primary Category',  'Secondary Category','Tertiary Category',  'Key words','Site Favorite','Legacy URL', 'URL Link', 'Content Type', 'Content Extension', 'Original Pub Date', 'System Pub Date', 'Next Review Date', 'Content Owner ID','Content Owner Name', 'Author First Name', 'Author Last Name','Reviewer First Name', 'Reviewer Last Name']]
metadata_required.head()

,content_id,Content Title,Content Description,Content State,SBU Code,SBU Name,Primary Category,Secondary Category,Tertiary Category,Key words,...,Content Extension,Original Pub Date,System Pub Date,Next Review Date,Content Owner ID,Content Owner Name,Author First Name,Author Last Name,Reviewer First Name,Reviewer Last Name
0,386,Cargo Commodity Guide,This is the Chubb Ocean Cargo Commodity Guide ...,Live,ci,Commercial Insurance,Strategy,Underwriting Guidelines,Strategies / Guidelines,"Commodity Guidelines, commodity codes, commodi...",...,NaN,08/08/2002,03/10/2020,12/01/2099,om,Ocean Cargo,Peter,Aitken,Lori,Hummel
1,393,Cargo Commodity Classification Table,Ocean Cargo Commodity Classification Table,Live,ci,Commercial Insurance,Strategy,Underwriting Guidelines,Strategies / Guidelines,Cargo Commodity Classification Table,...,.pdf,08/05/2003,03/10/2020,08/01/2015,om,Ocean Cargo,Peter,Aitken,Lori,Hummel
2,395,Life Science New Business Application,This application allows the applicant to enter...,Live,ci,Commercial Insurance,Applications,No Secondary Category,No Tertiary Category,New Business Application,...,.doc,10/13/2009,04/09/2020,10/13/2017,li,Life Sciences,Phil,Fiscus,Veronica,Somarriba
4,405,Life Sciences Glossary,Life Sciences Glossary - Terms and definitions...,Live,ci,Commercial Insurance,Applications,No Secondary Category,No Tertiary Category,"Glossary, definitions",...,.pdf,10/03/2001,01/29/2020,03/01/2008,li,Life Sciences,Jill,Wadlund,Kathleen,Ballantyne
6,406,Life Sciences Renewal Business Application,This application is a word document that allow...,Live,ci,Commercial Insurance,Applications,No Secondary Category,No Tertiary Category,Renewal Application,...,.docx,05/29/2019,01/29/2020,05/29/2020,li,Life Sciences,Jill,Wadlund,Kathleen,Ballantyne


In [95]:
master_df = df.merge(metadata_required,on='content_id',how='left')
master_df.shape

(12, 30)

In [96]:
master_df.columns

Index(['submission_doc', 'SIC_Num', 'lob', 'Relation', 'filename',
       'extracted_text', 'content_id', 'Content Title', 'Content Description',
       'Content State', 'SBU Code', 'SBU Name', 'Primary Category',
       'Secondary Category', 'Tertiary Category', 'Key words', 'Site Favorite',
       'Legacy URL', 'URL Link', 'Content Type', 'Content Extension',
       'Original Pub Date', 'System Pub Date', 'Next Review Date',
       'Content Owner ID', 'Content Owner Name', 'Author First Name',
       'Author Last Name', 'Reviewer First Name', 'Reviewer Last Name'],
      dtype='object')

In [97]:
master_df['Content Description'] = master_df['Content Description'].fillna('Risk Class Determination for CI Property')
master_df.loc[:,['SIC_Num','lob','Relation','content_id','extracted_text','submission_doc','Content Title']]

,SIC_Num,lob,Relation,content_id,extracted_text,submission_doc,Content Title
0,8733,General Liability,Exposures,70252,The following Life Sciences SIC Codes are Low ...,P318091211495676.pdf,"Life Sciences Equipment Breakdown: Authority, ..."
1,8733,Property,Exposures,70252,The following Life Sciences SIC Codes are Low ...,P318091211495676.pdf,"Life Sciences Equipment Breakdown: Authority, ..."
2,8733,General Liability,CRFs,63069,"Laboratories – SIC 8071, 8072, 8733, 8734, 873...",P318091211495676.pdf,Life Sciences Property Underwriting Guide
3,8733,Property,CRFs,63069,"Laboratories – SIC 8071, 8072, 8733, 8734, 873...",P318091211495676.pdf,Life Sciences Property Underwriting Guide
4,8733,General Liability,Exposures,63069,Primary Underwriting Considerations • Microbio...,P318091211495676.pdf,Life Sciences Property Underwriting Guide
5,8733,Property,Exposures,63069,Primary Underwriting Considerations • Microbio...,P318091211495676.pdf,Life Sciences Property Underwriting Guide
6,SIC Agnostic,Property,Risk Class,76207,When determining the appropriate risk class fo...,P318091211495676.pdf,NaN
7,8733,Property,Limits,66176,Description: \nProperty/Business Income Global...,P318091211495676.pdf,Suite of Enhancements for Life Sciences Firms
8,8733,General Liability,Guidelines,66176,Description:\nIncrease non-owned watercraft in...,P318091211495676.pdf,Suite of Enhancements for Life Sciences Firms
9,SIC Agnostic,General Liability,Underwriting Strategy,2435,A Privacy exclusion or an Access or Disclosure...,P318091211495676.pdf,Access and Disclosure and Privacy Strategy (CI)


In [90]:

master_df

,submission_doc,SIC_Num,lob,Relation,filename,extracted_text,content_id,Content Title,Content Description,Content State,...,Content Extension,Original Pub Date,System Pub Date,Next Review Date,Content Owner ID,Content Owner Name,Author First Name,Author Last Name,Reviewer First Name,Reviewer Last Name
0,P318091211495676.pdf,8733,General Liability,Exposures,rx70252.pdf,The following Life Sciences SIC Codes are Low ...,70252,"Life Sciences Equipment Breakdown: Authority, ...",This memo provides guidance on how to process ...,Live,...,.pdf,01/24/2020,01/29/2020,01/22/2021,li,Life Sciences,Frank,Goudsmit,Kathleen,Ballantyne
1,P318091211495676.pdf,8733,Property,Exposures,rx70252.pdf,The following Life Sciences SIC Codes are Low ...,70252,"Life Sciences Equipment Breakdown: Authority, ...",This memo provides guidance on how to process ...,Live,...,.pdf,01/24/2020,01/29/2020,01/22/2021,li,Life Sciences,Frank,Goudsmit,Kathleen,Ballantyne
2,P318091211495676.pdf,8733,General Liability,CRFs,rx63069.pdf,"Laboratories – SIC 8071, 8072, 8733, 8734, 873...",63069,Life Sciences Property Underwriting Guide,Life Sciences Property Underwriting Guidelines,Live,...,.pdf,08/19/2015,09/30/2020 18:54:14,09/30/2021,li,Life Sciences,Frank,Goudsmit,Kati,Ballantyne
3,P318091211495676.pdf,8733,Property,CRFs,rx63069.pdf,"Laboratories – SIC 8071, 8072, 8733, 8734, 873...",63069,Life Sciences Property Underwriting Guide,Life Sciences Property Underwriting Guidelines,Live,...,.pdf,08/19/2015,09/30/2020 18:54:14,09/30/2021,li,Life Sciences,Frank,Goudsmit,Kati,Ballantyne
4,P318091211495676.pdf,8733,General Liability,Exposures,rx63069.pdf,Primary Underwriting Considerations • Microbio...,63069,Life Sciences Property Underwriting Guide,Life Sciences Property Underwriting Guidelines,Live,...,.pdf,08/19/2015,09/30/2020 18:54:14,09/30/2021,li,Life Sciences,Frank,Goudsmit,Kati,Ballantyne
5,P318091211495676.pdf,8733,Property,Exposures,rx63069.pdf,Primary Underwriting Considerations • Microbio...,63069,Life Sciences Property Underwriting Guide,Life Sciences Property Underwriting Guidelines,Live,...,.pdf,08/19/2015,09/30/2020 18:54:14,09/30/2021,li,Life Sciences,Frank,Goudsmit,Kati,Ballantyne
6,P318091211495676.pdf,SIC Agnostic,Property,Risk Class,76207.pdf,When determining the appropriate risk class fo...,76207,Risk Class Determination for CI Property,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,P318091211495676.pdf,8733,Property,Limits,66176.pdf,Description: \nProperty/Business Income Global...,66176,Suite of Enhancements for Life Sciences Firms,This memo recaps coverage and limits we should...,Live,...,.pdf,12/20/2019,03/29/2020,12/20/2020,li,Life Sciences,Frank,Goudsmit,Kati,Ballantyne
8,P318091211495676.pdf,8733,General Liability,Guidelines,66176.pdf,Description:\nIncrease non-owned watercraft in...,66176,Suite of Enhancements for Life Sciences Firms,This memo recaps coverage and limits we should...,Live,...,.pdf,12/20/2019,03/29/2020,12/20/2020,li,Life Sciences,Frank,Goudsmit,Kati,Ballantyne
9,P318091211495676.pdf,SIC Agnostic,General Liability,Underwriting Strategy,2435.pdf,A Privacy exclusion or an Access or Disclosure...,2435,Access and Disclosure and Privacy Strategy (CI),This is the updated worldwide CCI Privacy Stra...,Live,...,.pdf,04/08/2019,08/24/2020 13:37:28,06/01/2025,it,Technology (INT),Bonnie,Caluri,Daniel,Reiner


# Create Neo4j Connection

In [72]:
#creating connection
uri = "bolt://neo4j@localhost:7687"
remote_graph = Graph(uri, user="neo4j", password="password")

In [69]:
#setting up unique constraints
try:
    remote_graph.schema.create_uniqueness_constraint('SBU','SBU_Name')
except:
    pass


#setting up unique constraints
try:
    remote_graph.schema.create_uniqueness_constraint('Content_State','Content_State')
except:
    pass


#setting up unique constraints
try:
    remote_graph.schema.create_uniqueness_constraint('Content_State','Content_State')
except:
    pass


try:
    remote_graph.schema.create_uniqueness_constraint('Document','content_id')
except:
    pass

try:
    remote_graph.schema.create_uniqueness_constraint('Document','Document')
except:
    pass


try:
    remote_graph.schema.create_uniqueness_constraint('Content_Owner','Content_Owner_ID')
except:
    pass

try:
    remote_graph.schema.create_uniqueness_constraint('Keywords','Key_words')
except:
    pass

try:
    remote_graph.schema.create_uniqueness_constraint('Original_Published_Date','Original_Pub_Date')
except:
    pass

try:
    remote_graph.schema.create_uniqueness_constraint('Primary_Category','Primary_Category')
except:
    pass

try:
    remote_graph.schema.create_uniqueness_constraint('Secondary_Category','Secondary_Category')
except:
    pass

try:
    remote_graph.schema.create_uniqueness_constraint('Tertiary_Category','Tertiary_Category')
except:
    pass

try:
    remote_graph.schema.create_uniqueness_constraint('Additional_Links','additional_links')
except:
    pass

try:
    remote_graph.schema.create_uniqueness_constraint('LOB','Line of Business')
except:
    pass

try:
    remote_graph.schema.create_uniqueness_constraint('Submission','submission_doc')
except:
    pass


# Define Relationship types

In [101]:
CONTENT_OWNED_BY = Relationship.type("CONTENT_OWNED_BY")
HAVING_TITLE = Relationship.type("HAVING_TITLE")
CONTENT_STATUS = Relationship.type("CONTENT_STATUS")
#HAVING_BASIS = Relationship.type("HAVING_BASIS")
HAVING_SIC_Num = Relationship.type("HAVING_SIC_Num")
REFFERNCES = Relationship.type("REFFERNCES")
Line_of_Business = Relationship.type("LOB")
#RELATION_TYPE = Relationship.type("RELATION_TYPE")
CONTEXT = Relationship.type("CONTEXT")
#Connection_TYPE = Relationship.type("bigram")

matcher = NodeMatcher(remote_graph)

#change the number of records accordingly .
for index, row in tqdm(master_df[:15000].iterrows()):
    
#     ## LEVEL 1 COMMERCIAL INSURANCE
#     sbu = Node("SBU Name", SBU = row['SBU Name'])
#     sbu.__primarylabel__ = "SBU"
#     sbu.__primarykey__ = "SBU"
#     sbu.__primaryvalue__ = "SBU"
    
    
#     ## LEVEL 2 LIVE OR ARCHIVE
#     c_state = Node("Content_State Name", Content_State = row['Content State'])
#     c_state.__primarylabel__ = "Content_State"
#     c_state.__primarykey__ = "Content_State"
#     c_state.__primaryvalue__ = "Content_State"
#     remote_graph.merge(CONTENT_STATUS(sbu, c_state))
    
#      ## LEVEL 3 #CONTENT_OWNER, NAME TAG COMES from here
#     co = Node("Content Owner", Content_Owner_ID = row['Content Owner ID'], Name = row['Content Owner Name'])
#     co.__primarylabel__ = "Name"
#     co.__primarykey__ = "Content_Owner_ID"
#     co.__primaryvalue__ = "Name"
#     remote_graph.merge(CONTENT_OWNED_BY(c_state, co))
    
    
    ## LEVEl 4 SIC LEVEL
#     sic_num = Node("SIC_Num", SIC_Num = row['SIC_Num'])
#     sic_num.__primarylabel__ = "SIC_Num"
#     sic_num.__primarykey__ = "SIC_Num"
#     sic_num.__primaryvalue__ = "SIC_Num"
#     remote_graph.merge(HAVING_SIC_Num(co,sic_num))
    
    ## LEVEl 5 LOB LEVEL
    lob = Node("LOB", LOB = row['lob'])
    lob.__primarylabel__ = "LOB"
    lob.__primarykey__ = "LOB"
    lob.__primaryvalue__ = "LOB"
    #remote_graph.merge(Line_of_Business(sic_num,lob))
    

    
        ## LEVEL 6 DOCUMENT LEVEL
    doc = Node("Document", Content_Id = row['content_id'],Original_Published_Date = row['Original Pub Date'],Content_Title = row['Content Title'], Content_URL = row['URL Link'],Description=row['Content Description'])
    doc.description = "Description"
    doc.__primarylabel__ = "Content_URL"
    doc.__primarykey__ = "Content_Id"
    doc.__primaryvalue__ = "Content_Title"
    doc.published_date = "Original_Published_Date"
    rel = row['Relation']
    if rel == 'No Relation':
        pass
    else:
        LOB = Relationship.type(rel)
        remote_graph.merge(LOB(lob, doc))
        
    #doc1 = Node("Context", Content_Id = row['content_id'],Original_Published_Date = row['Original Pub Date'],Content_Title = row['Content Title'], Content_URL = row['URL Link'],Description=row['Content Description'])
    #doc.description = "Description"
    
    
    ctx = Node("Context", Context = row['extracted_text'])
    #ctx.description = "Description"
    ctx.__primarylabel__ = "Context"
    ctx.__primarykey__ = "Context"
    ctx.__primaryvalue__ = "Context"
    #rel = row['bigram']
    #rel="context"
    #if rel == None:
    #    pass
    #else:
    #Context = Relationship.type(rel)
    #remote_graph.merge(CONTEXT(doc,ctx ))
    rel = row['Relation']
    if rel == 'No Relation':
        pass
    else:
        LOB = Relationship.type(rel)
        remote_graph.merge(LOB(doc, ctx))
  
    
    sub = Node("Submission", Submission = row['submission_doc'])
    #ctx.description = "Description"
    sub.__primarylabel__ = "Submission"
    sub.__primarykey__ = "Submission"
    sub.__primaryvalue__ = "Submission"
    rel = row['SIC_Num']
    LOB = Relationship.type(rel)
    remote_graph.merge(LOB(sub,doc))
    
    remote_graph.merge(Line_of_Business(sub,lob))

    
#     ## LEVEl 6 DOCUMENT LEVEL
#     if row['additional_links'] != None: 
#         links = Node("Additional_Links", Additional_Links = row['additional_links'])
#         links.__primarylabel__ = "Additional_Links"
#         links.__primarykey__ = "Additional_Links"
#         links.__primaryvalue__ = "Additional_Links"
#         remote_graph.merge(REFFERNCES(doc, links))
#     else:
#         pass
#     try :
#         links.merge(HAVING_URLS(doc, links))
#     except AttributeError:
#         pass
      

12it [00:00, 16.70it/s]


# Delete nodes

In [100]:
remote_graph.delete_all()